# Knowledge distillation

## 1) UC1: Anomaly Detection

In [1]:
import os
os.chdir('/home/benfenati/code/tle-supervised/')

In [2]:
import torch
import pandas as pd
import numpy as np

from Algorithms.models_audio_mae import audioMae_vit_base
from Algorithms.models_audio_mae_evaluate import audioMae_vit_base_evaluate

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from utils import *
from util.engine_pretrain import evaluate
from plot_anomaly import compute_threshold_accuracy

import datetime

from Datasets.AnomalyDetection_SS335.get_dataset import get_dataset as get_dataset_ss335

In [3]:
device = "cuda:2"
dir = "/home/benfenati/code/Datasets/SHM/AnomalyDetection_SS335/"
window_size = 1190
lr = 0.25e-2
total_epochs = 1

### Teacher model

In [4]:
device = torch.device(device)
teacher = audioMae_vit_base_evaluate(norm_pix_loss=False)
teacher.to(device)
checkpoint = torch.load(f"/home/benfenati/code/shm/checkpoints/checkpoint-pretrain_all-200.pth", map_location='cpu')
checkpoint_model = checkpoint['model']
msg = teacher.load_state_dict(checkpoint_model, strict=False)

params, size = get_model_info(teacher)
print("N. params = {}; Size = {:.3f}".format(params, size))

N. params = 36116189; Size = 137.772


### Student model

In [5]:
embed_dim = 384 # 384, 768(original)
decoder_embed_dim = 256 # 512(original)
student = audioMae_vit_base_evaluate(embed_dim=embed_dim, decoder_embed_dim=decoder_embed_dim, norm_pix_loss=False)
student.to(device)

params, size = get_model_info(student)
print("N. params = {}; Size = {:.3f}".format(params, size))

N. params = 9179101; Size = 35.015


### Training

In [6]:
starting_date = datetime.date(2019,5,22) 
num_days = 7
print("Creating Training Dataset")
dataset = get_dataset_ss335(dir, starting_date, num_days, sensor = 'S6.1.3', time_frequency = "frequency", windowLength = window_size)
sampler_train = torch.utils.data.RandomSampler(dataset)
data_loader_train = torch.utils.data.DataLoader(
    dataset, sampler=sampler_train,
    batch_size=64,
    num_workers=1,
    pin_memory='store_true',
    drop_last=True,
)
device = torch.device(device)
torch.manual_seed(0)
np.random.seed(0)

optimizer = optim.Adam(student.parameters(), lr=0.001, weight_decay=1e-6)
loss_fn_1 = nn.L1Loss()
loss_fn_2 = nn.L1Loss()
loss_fn_3 = nn.MSELoss()

teacher.eval()

b = 0.5

best_loss = 100000000
best_epoch = 0

for epoch in range(total_epochs):

    student.train()
    train_loss = 0
    counter = 0
    for samples, targets in data_loader_train:
        samples = samples.to(device, non_blocking=True)
        targets = targets.to(device, non_blocking=True)
    
        
        optimizer.zero_grad()
        with torch.cuda.amp.autocast():
            loss_student, pred_student, _ = student(samples, mask_ratio=0.8)

        with torch.no_grad() and torch.cuda.amp.autocast():
            teacher.eval()
            loss_teacher, pred_teacher, _ = teacher(samples, mask_ratio=0.8)
        
        
        loss_1 = loss_student
        loss_2 = loss_fn_3(pred_student, pred_teacher)
        
        loss = b*loss_1 + (1-b)*loss_2

        loss.backward()
        optimizer.step()

        train_loss += loss_student.item()
        # train_loss += loss.item()
        counter +=1

Creating Training Dataset
Loading AnomalyDetection dataset 

### Testing

In [9]:
model_to_evaluate = teacher
who = "teacher"

### Creating Testing Dataset for Normal Data
starting_date = datetime.date(2019,5,10)
num_days = 4
print("Creating Testing Dataset -- Normal")
dataset = get_dataset_ss335(dir, starting_date, num_days, sensor = 'S6.1.3', time_frequency = "frequency", windowLength = window_size)
data_loader_test_normal = torch.utils.data.DataLoader(
    dataset, shuffle=False,
    batch_size=1,
    num_workers=1,
    pin_memory='store_true',
    drop_last=True,
)
losses_normal = evaluate(data_loader_test_normal, model_to_evaluate, device)
df = pd.DataFrame.from_dict(losses_normal)
df.to_csv(f'Results/masked_{window_size}samples_normal_{who}.csv', index = False, header = True)
    
### Creating Testing Dataset for Anomaly Data
starting_date = datetime.date(2019,4,17) 
num_days = 4
print("Creating Testing Dataset -- Anomaly")
dataset = get_dataset_ss335(dir, starting_date, num_days, sensor = 'S6.1.3', time_frequency = "frequency", windowLength = window_size)
data_loader_test_anomaly = torch.utils.data.DataLoader(
    dataset, shuffle=False,
    batch_size=1,
    num_workers=1,
    pin_memory='store_true',
    drop_last=True,
)
losses_anomaly = evaluate(data_loader_test_anomaly, model_to_evaluate, device)
df = pd.DataFrame.from_dict(losses_anomaly)
df.to_csv(f'Results/masked_{window_size}samples_anomaly_{who}.csv', index = False, header = True)

directory = "/home/benfenati/code/tle-supervised/Results/"
acc_enc = []
sens_enc = []
spec_enc = []

for dim_filtering in [15,30,60,120, 240]:
        print(f"Dim {dim_filtering}")
        print(f"Autoencoder")
        data_normal = pd.read_csv(directory + f"masked_{window_size}samples_normal_{who}.csv")
        data_anomaly = pd.read_csv(directory + f"masked_{window_size}samples_anomaly_{who}.csv")
        spec, sens, acc = compute_threshold_accuracy(data_anomaly.values, data_normal.values, None, min, max, only_acc = 1, dim_filtering = dim_filtering)
        acc_enc.append(acc*100)
        sens_enc.append(sens*100)
        spec_enc.append(spec*100)

Creating Testing Dataset -- Normal
Loading AnomalyDetection dataset Test:  [   0/5279]  eta: 0:09:44  loss: 1.2760 (1.2760)  mae1: 1.2760 (1.2760)  time: 0.1107  data: 0.0937  max mem: 0
Test:  [  10/5279]  eta: 0:01:15  loss: 1.2758 (1.2753)  mae1: 1.2758 (1.2753)  time: 0.0142  data: 0.0086  max mem: 0
Test:  [  20/5279]  eta: 0:00:53  loss: 1.2723 (1.2749)  mae1: 1.2723 (1.2749)  time: 0.0051  data: 0.0001  max mem: 0
Test:  [  30/5279]  eta: 0:00:46  loss: 1.2764 (1.2799)  mae1: 1.2764 (1.2799)  time: 0.0059  data: 0.0001  max mem: 0
Test:  [  40/5279]  eta: 0:00:41  loss: 1.2824 (1.2797)  mae1: 1.2824 (1.2797)  time: 0.0057  data: 0.0001  max mem: 0
Test:  [  50/5279]  eta: 0:00:40  loss: 1.2803 (1.2800)  mae1: 1.2803 (1.2800)  time: 0.0059  data: 0.0001  max mem: 0
Test:  [  60/5279]  eta: 0:00:38  loss: 1.2802 (1.2805)  mae1: 1.2802 (1.2805)  time: 0.0062  data: 0.0001  max mem: 0
Test:  [  70/5279]  eta: 0:00:36  loss: 1.2802 (1.2804)  mae1: 1.2802 (1.2804)  time: 0.0056  data:

In [8]:
directory = "/home/benfenati/code/shm/Results/"
acc_enc = []
sens_enc = []
spec_enc = []

for dim_filtering in [15,30,60,120, 240]:
        print(f"Dim {dim_filtering}")
        print(f"Autoencoder")
        data_normal = pd.read_csv(directory + f"masked_{window_size}samples_normal-{who}.csv")
        data_anomaly = pd.read_csv(directory + f"masked_{window_size}samples_anomaly_{who}.csv")
        spec, sens, acc = compute_threshold_accuracy(data_anomaly.values, data_normal.values, None, min, max, only_acc = 1, dim_filtering = dim_filtering)
        acc_enc.append(acc*100)
        sens_enc.append(sens*100)
        spec_enc.append(spec*100)

Dim 15
Autoencoder
Sensitivity: 1.0 Spcificity: 1.0 Accuracy: 1.0
Dim 30
Autoencoder
Sensitivity: 1.0 Spcificity: 1.0 Accuracy: 1.0
Dim 60
Autoencoder
Sensitivity: 1.0 Spcificity: 1.0 Accuracy: 1.0
Dim 120
Autoencoder
Sensitivity: 1.0 Spcificity: 1.0 Accuracy: 1.0
Dim 240
Autoencoder
Sensitivity: 1.0 Spcificity: 0.999810570183747 Accuracy: 0.9999130888232227
